<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/01_supervised/09_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoML Framework

### Nota Introduttiva
Questo notebook è la traduzione dell'articolo [**AutoML Frameworks in R & Python**](https://www.r-bloggers.com/automl-frameworks-in-r-python/). Noi tratteremo solo la parte Python.

Ho aggiunto questo notebook per mostare degli strumenti che stanno prendendo molto piede, qualche anno fa si parlava addirittura di fine dei data scientist, e per far vedere che nonostante siano un grande aiuto per chi fa questo lavoro non basata "*prendere i dati e buttarli dentro*" per avere un buon modello.

In particolare vedremo [H2O](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html)


**Il dataset utilizzato sarà Iris.**

## Cos'è AutoML

Negli ultimi anni, AutoML o Machine Learning automatizzato sono diventati molto popolari.

I giganti della tecnologia come Google, Amazon e Microsoft hanno iniziato a offrire strumenti AutoML.

C'è ancora una divisione tra i data scientist quando si tratta di AutoML. Alcuni temono che sarà una minaccia per il loro lavoro o peggio far chiudere interi rami aziendali. Altri lo vedono come uno strumento che potrebbero sfruttare per compiti non critici o per rafforzare le loro intuizioni.

**Pro:**

* **Risparmio di tempo**: è uno strumento di prototipazione rapida in logica "*smart & dirty*". Se non stai lavorando su attività critiche, puoi utilizzare AutoML per fare il lavoro per te mentre ti concentri su attività che richiedono maggiorre concentrazione.
* **Benchmarking**: costruire un modello ML / DL è divertente. Ma come fai a sapere che il modello che hai è il migliore? O devi dedicare molto tempo alla costruzione di modelli iterativi o chiedere al tuo collega di costruirne uno e confrontarlo. L'altra opzione è utilizzare AutoML per confrontare i tuoi.

**Contro:**

* La maggior parte dei modelli di intelligenza artificiale, in particolare le reti neurali, sono **scatole nere**. Simile è il caso di questi framework AutoML. Se non si comprende bene cosa si sta facendo, potrebbe essere catastrofico.
* AutoML viene commercializzato come uno strumento per citizen data scientist e per chi non fa data science come principale occupazione. Questa è una brutta mossa. **Senza capire come funziona un modello e usarlo ciecamente per prendere decisioni potrebbe essere disastroso.**


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [0]:
data = load_iris()

In [0]:
iris = pd.DataFrame({'sepal length': data.data[:,0],
  'sepal width': data.data[:,1],
  'petal length': data.data[:,2],
  'petal width': data.data[:,3], 'species': data.target})

In [0]:
train, test = train_test_split(iris, test_size = 0.25, random_state = 42)

## H2O

<br>

![h20](https://iamnagdev.files.wordpress.com/2020/04/1s6ke_nwoge5m7ok1onsjsq.png?w=351&h=375&fit=351%2C375&resize=351%2C375)

<br>

[Image Credits](https://www.r-bloggers.com/automl-frameworks-in-r-python/)

<br>

H2O Offre modelli di ensemble ML, deep learning e altri. Sebbene sia scritto in Java, offre connettori per R e Python tramite API. [Una delle migliori caratteristiche è il "*tempo di arresto*", in cui è possibile impostare per quanto tempo si vuole allenare il modello. Caratteristica analoga è la scelta del numero di modelli da provare](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html).

Restituisce una leaderboard di modelli è [possibile ordinarla e scegliere secondo diverse metriche](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/sort_metric.html).

In [0]:
# installazione
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [0]:
# load python libraries
import h2o
from h2o.automl import H2OAutoML

# start cluster
h2o.init()

In [0]:
# convert to h2o frame
traindf = h2o.H2OFrame(train)
testdf = h2o.H2OFrame(test)
 
y = "species"
x = list(traindf.columns)
x.remove(y)
 
# create df to factors
traindf[y] = traindf[y].asfactor()
testdf[y] = testdf[y].asfactor()

In [0]:
#run automl
aml = H2OAutoML(max_runtime_secs = 60)
aml.train(x = x, y = y, training_frame = traindf)

In [0]:
 # view leader board
aml.leaderboard

In [0]:
aml.leader

In [0]:
# do pridiction and convert it to a data frame
predict = aml.predict(testdf)
p = predict.as_data_frame()
 
# convert to pandas dataframe
data = {'actual': test.species, 'Ypredict': p['predict'].tolist()}
 
df = pd.DataFrame(data, columns = ['actual','Ypredict'])
 
# create a confusion matrix and print results
confusion_matrix = pd.crosstab(df['actual'], df['Ypredict'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

In [0]:
# close h2o connection
h2o.shutdown(prompt = False)